In [1]:
import h3
import polars as pl

import duckdb

# duckdb.install_extension('spatial')
# duckdb.install_extension('h3', repository='community')

duckdb.load_extension('spatial')
duckdb.load_extension('h3')

In [2]:
# Join spatially on points using radius (more accurate, but with additional conversions)

relation = duckdb.sql(
    """
    WITH stations_data AS (
        SELECT station_id, date, lat, lon, hour, traffic FROM read_parquet('../for_participants/data_parquet/traffic_train.parquet')
    ),
    signals AS (
        SELECT * FROM read_parquet('../for_participants/data_parquet/signals.parquet')
    ),
    distict_stations AS (
        SELECT DISTINCT station_id, lat, lon, FROM stations_data
    ),
    distinct_signals_locations AS (
        SELECT DISTINCT h3res13 FROM signals
    ),
    distinct_join_keys AS (
        SELECT station_id, h3res13
        FROM distict_stations
        JOIN distinct_signals_locations ON ST_Distance_Sphere(
            ST_POINT(
                distict_stations.lon,
                distict_stations.lat
            ),
            ST_POINT(
                h3_cell_to_lng(distinct_signals_locations.h3res13),
                h3_cell_to_lat(distinct_signals_locations.h3res13)
            )
        ) < 1000 -- 1km
    ),
    signals_in_station_range AS (
        SELECT *
        FROM stations_data
        JOIN distinct_join_keys 
        ON stations_data.station_id = distinct_join_keys.station_id
        JOIN signals
        ON stations_data.date = signals.signal_date
        AND distinct_join_keys.h3res13 = signals.h3res13
    )
    SELECT 
       station_id,
       date,
       hour,
       traffic,
       SUM(ctn_per_day) AS ctn_per_day_in_station_range, 
       SUM(ctn_0) AS cnt_0_in_station_range,
       SUM(ctn_1) AS cnt_1_in_station_range, 
       SUM(ctn_2) AS cnt_2_in_station_range,
       SUM(ctn_3) AS cnt_3_in_station_range, 
       SUM(ctn_4) AS cnt_4_in_station_range, 
       SUM(ctn_5) AS cnt_5_in_station_range,
       SUM(ctn_6) AS cnt_6_in_station_range, 
       SUM(ctn_7) AS cnt_7_in_station_range,
       SUM(ctn_8) AS cnt_8_in_station_range, 
       SUM(ctn_9) AS cnt_9_in_station_range,
       SUM(ctn_10) AS cnt_10_in_station_range, 
       SUM(ctn_11) AS cnt_11_in_station_range, 
       SUM(ctn_12) AS cnt_12_in_station_range, 
       SUM(ctn_13) AS cnt_13_in_station_range,
       SUM(ctn_14) AS cnt_14_in_station_range, 
       SUM(ctn_15) AS cnt_15_in_station_range,
       SUM(ctn_16) AS cnt_16_in_station_range, 
       SUM(ctn_17) AS cnt_17_in_station_range, 
       SUM(ctn_18) AS cnt_18_in_station_range, 
       SUM(ctn_19) AS cnt_19_in_station_range, 
       SUM(ctn_20) AS cnt_20_in_station_range, 
       SUM(ctn_21) AS cnt_21_in_station_range, 
       SUM(ctn_22) AS cnt_22_in_station_range, 
       SUM(ctn_23) AS cnt_23_in_station_range, 
    FROM signals_in_station_range
    GROUP BY 1, 2, 3, 4
    """
)

In [3]:
r = relation.pl()

In [4]:
r

station_id,date,hour,traffic,ctn_per_day_in_station_range,cnt_0_in_station_range,cnt_1_in_station_range,cnt_2_in_station_range,cnt_3_in_station_range,cnt_4_in_station_range,cnt_5_in_station_range,cnt_6_in_station_range,cnt_7_in_station_range,cnt_8_in_station_range,cnt_9_in_station_range,cnt_10_in_station_range,cnt_11_in_station_range,cnt_12_in_station_range,cnt_13_in_station_range,cnt_14_in_station_range,cnt_15_in_station_range,cnt_16_in_station_range,cnt_17_in_station_range,cnt_18_in_station_range,cnt_19_in_station_range,cnt_20_in_station_range,cnt_21_in_station_range,cnt_22_in_station_range,cnt_23_in_station_range
str,date,i64,i64,"decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]"
"""5398""",2024-04-14,0,214,1259,92,92,88,96,91,143,206,182,139,158,115,184,202,207,285,113,136,91,76,92,28,102,97,94
"""5398""",2024-04-04,0,195,2067,23,60,26,61,51,276,347,367,173,282,209,213,172,222,278,303,157,235,156,143,101,113,37,28
"""5398""",2024-03-02,1,154,1899,49,30,39,15,49,51,122,191,255,323,445,281,250,354,277,190,78,182,98,165,73,145,75,52
"""5398""",2024-04-16,1,123,1763,46,8,47,55,38,120,285,322,283,202,141,161,249,160,122,219,301,119,114,84,16,26,63,56
"""5398""",2024-03-08,23,340,2440,26,14,42,40,83,267,437,238,203,288,305,301,339,373,535,360,341,298,143,145,96,70,30,15
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""3365""",2024-03-28,15,3655,1720,18,21,23,65,85,208,476,258,304,164,302,216,194,102,211,168,157,172,181,124,77,57,15,2
"""3365""",2024-04-26,18,3380,1491,30,27,19,32,80,209,2,182,280,165,277,215,201,236,250,155,174,154,128,160,62,45,26,38
"""3365""",2024-04-18,18,3515,1310,19,15,15,21,43,163,247,242,134,63,148,106,100,88,184,252,180,93,70,43,28,21,41,20


In [2]:
buildings = pl.read_parquet('../for_participants/data_parquet/buildings.parquet')

In [26]:
buildings['h3res13'].unique()

h3res13
str
"""8d1f53532b51a3f"""
"""8d1f53c9291c8ff"""
"""8d1f53d9b170d7f"""
"""8d1f535137526ff"""
"""8d1f53cd363533f"""
…
"""8d1f523491ae33f"""
"""8d1f53c89515a7f"""
"""8d1f5226171c8bf"""


In [4]:
h3.get_resolution("8d1f53cd3299dbf")

13

In [5]:
import osmnx as ox
import networkx as nx


def download_city_network(city_name, network_type='walk'):
    """Download an entire city network once"""
    print(f"Downloading {network_type} network for {city_name}...")
    G = ox.graph_from_place(city_name, network_type=network_type)
    return G

def calculate_walking_distance(G, lat1, lon1, lat2, lon2):  
    # Get the nearest nodes to your points
    origin_node = ox.distance.nearest_nodes(G, lon1, lat1)
    destination_node = ox.distance.nearest_nodes(G, lon2, lat2)
    
    # Calculate the shortest path
    try:
        route = nx.shortest_path(G, origin_node, destination_node, weight='length')
        # Calculate the total distance of the route in meters
        distance = int(sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length')))
        return distance
    except nx.NetworkXNoPath:
        return "No path found between these points."

In [6]:
g = download_city_network('Warsaw, Poland')

In [27]:
calculate_walking_distance(
    g,
    *h3.cell_to_latlng("8d1f53cd329d6ff"),
    *h3.cell_to_latlng("8d1f5226171c8bf")
)

AttributeError: module 'osmnx' has no attribute 'utils_graph'

In [38]:
origin_node = ox.distance.nearest_nodes(g, 21.261341934369394, 52.215532347089564)
destination_node = ox.distance.nearest_nodes(g, 20.944292244485577, 52.19985141666157)

In [39]:
route = nx.shortest_path(g, origin_node, destination_node, weight='length')

In [40]:
route

[5481493545,
 11393318402,
 9278944752,
 5481493552,
 7099944634,
 8541160502,
 8541160501,
 8875507838,
 5481493588,
 5481493641,
 9278944761,
 8541160508,
 1387201032,
 1475024130,
 920226473,
 9011990959,
 1387201029,
 6173483263,
 3139759394,
 1161317422,
 1161317666,
 1161317399,
 1161317513,
 920226477,
 920226478,
 9911418087,
 1475018138,
 9911418073,
 920224912,
 9911419608,
 1110713683,
 9911418064,
 1110713671,
 1110713679,
 9911418046,
 11980819562,
 4390229639,
 2766970508,
 8873154606,
 1110713658,
 8199186027,
 1161317546,
 1161317485,
 1161317633,
 3205155380,
 921139572,
 8199154812,
 1110713615,
 4899842583,
 921139576,
 9911417961,
 9911546334,
 9911417803,
 921139578,
 3857690821,
 1110713608,
 3139784345,
 3139784346,
 1157619888,
 1157619513,
 1161317605,
 1161317663,
 1161317556,
 1161317355,
 1157619776,
 6113854079,
 1157619259,
 987127635,
 1157619988,
 1157619621,
 1145577187,
 8245207352,
 1145577178,
 6113843922,
 1145577142,
 2640749008,
 10315038805,
 114

In [36]:
g.nodes[origin_node]

{'y': 52.1494991, 'x': 21.2582759, 'street_count': 3}

In [9]:
import osmnx


In [31]:
h3.cell_to_latlng("8d1f53cd329d6ff")

(52.215532347089564, 21.261341934369394)

In [32]:
h3.cell_to_latlng("8d1f5226171c8bf")

(52.19985141666157, 20.944292244485577)

In [23]:
origin_node

3839199511

In [34]:
ox.distance.nearest_nodes?

Signature:
ox.distance.nearest_nodes(
    G: 'nx.MultiDiGraph',
    X: 'float | Iterable[float]',
    Y: 'float | Iterable[float]',
    *,
    return_dist: 'bool' = False,
) -> 'int | npt.NDArray[np.int64] | tuple[int, float] | tuple[npt.NDArray[np.int64], npt.NDArray[np.float64]]'
Docstring:
Find the nearest node to a point or to each of several points.

If `X` and `Y` are single coordinate values, this function will return the
nearest node to that point. If `X` and `Y` are iterables of coordinate
values, it will return the nearest node to each point.

This function is vectorized: if you have many points to search for, pass
them in one call as numpy arrays (avoid using loops) to maximize runtime
speed. If the graph is projected, it uses a k-d tree for Euclidean nearest
neighbor search, which requires that scipy is installed as an optional
dependency. If the graph is unprojected, it uses a ball tree for haversine
nearest neighbor search, which requires that scikit-learn is installed as